In [1]:
# Load your usual SpaCy model (one of SpaCy English models)
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
#nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en')

In [2]:
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
#nlp = spacy.load('en')
nlp.add_pipe(coref, name='neuralcoref')


In [3]:
doc = nlp('My sister has a dog. She loves him.')
print(doc._.has_coref)
print(doc._.coref_clusters)
print(doc._.coref_resolved)

True
[My sister: [My sister, She], a dog: [a dog, him]]
My sister has a dog. My sister loves a dog.


In [4]:
from summarizer import Summarizer
from pprint import pprint

In [14]:
data = '''Peter Pan is a fictional character created by Scottish novelist and playwright J. M. Barrie. A free-spirited and mischievous young boy who can fly and never grows up, Peter Pan spends his never-ending childhood having adventures on the mythical island of Neverland as the leader of the Lost Boys, interacting with fairies, pirates, mermaids, Native Americans, and occasionally ordinary children from the world outside Neverland.

Peter Pan has become a cultural icon symbolizing youthful innocence and escapism. In addition to two distinct works by Barrie, The Little White Bird (1902, with chapters 13–18 published in Peter Pan in Kensington Gardens in 1906), and the West End stage play Peter Pan; or, the Boy Who Wouldn't Grow Up (1904, which expanded into the 1911 novel Peter and Wendy), the character has been featured in a variety of media and merchandise, both adapting and expanding on Barrie's works. These include the 1924 silent film, 1953 Disney animated film, a 2003 dramatic/live-action film, a television series and many other works.

Barrie commissioned a statue of Peter Pan by the sculptor George Frampton, which was erected overnight in Kensington Gardens on 30 April 1912 as a surprise to the children of London.[1] Six other statues have been cast from the original mould and displayed around the world. In 2002, Peter Pan featured on a series of UK postage stamps issued by the Royal Mail on the centenary of Barrie's creation of the character.'''

In [15]:
data

"Peter Pan is a fictional character created by Scottish novelist and playwright J. M. Barrie. A free-spirited and mischievous young boy who can fly and never grows up, Peter Pan spends his never-ending childhood having adventures on the mythical island of Neverland as the leader of the Lost Boys, interacting with fairies, pirates, mermaids, Native Americans, and occasionally ordinary children from the world outside Neverland.\n\nPeter Pan has become a cultural icon symbolizing youthful innocence and escapism. In addition to two distinct works by Barrie, The Little White Bird (1902, with chapters 13–18 published in Peter Pan in Kensington Gardens in 1906), and the West End stage play Peter Pan; or, the Boy Who Wouldn't Grow Up (1904, which expanded into the 1911 novel Peter and Wendy), the character has been featured in a variety of media and merchandise, both adapting and expanding on Barrie's works. These include the 1924 silent film, 1953 Disney animated film, a 2003 dramatic/live-ac

In [16]:
data = data.replace("\ufeff", "")
data[0:100]

'Peter Pan is a fictional character created by Scottish novelist and playwright J. M. Barrie. A free-'

In [17]:
model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
result = model(data, num_sentences=5, min_length=60)

In [19]:
full = ''.join(result)

In [20]:
pprint(result)

('Peter Pan is a fictional character created by Scottish novelist and '
 'playwright J. M. Barrie. A free-spirited and mischievous young boy who can '
 'fly and never grows up, Peter Pan spends his never-ending childhood having '
 'adventures on the mythical island of Neverland as the leader of the Lost '
 'Boys, interacting with fairies, pirates, mermaids, Native Americans, and '
 'occasionally ordinary children from the world outside Neverland. Peter Pan '
 'has become a cultural icon symbolizing youthful innocence and escapism. '
 'These include the 1924 silent film, 1953 Disney animated film, a 2003 '
 'dramatic/live-action film, a television series and many other works. In '
 '2002, Peter Pan featured on a series of UK postage stamps issued by the '
 "Royal Mail on the centenary of Barrie's creation of the character.")


In [21]:
len(data)

1467

In [22]:
len(full)

791

In [1]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = """Videos that say approved vaccines are dangerous and cause autism, cancer or infertility are among those that will be taken down, the company said.  The policy includes the termination of accounts of anti-vaccine influencers.  Tech giants have been criticised for not doing more to counter false health information on their sites.  In July, US President Joe Biden said social media platforms were largely responsible for people's scepticism in getting vaccinated by spreading misinformation, and appealed for them to address the issue.  YouTube, which is owned by Google, said 130,000 videos were removed from its platform since last year, when it implemented a ban on content spreading misinformation about Covid vaccines.  In a blog post, the company said it had seen false claims about Covid jabs "spill over into misinformation about vaccines in general". The new policy covers long-approved vaccines, such as those against measles or hepatitis B.  "We're expanding our medical misinformation policies on YouTube with new guidelines on currently administered vaccines that are approved and confirmed to be safe and effective by local health authorities and the WHO," the post said, referring to the World Health Organization."""

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)


/Users/manishajohnson/opt/anaconda3/envs/summarization-conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

ConnectionError: HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out.